In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1088311,2021-03-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1088312,2021-03-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1088313,2021-03-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1088314,2021-03-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
29371,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29373,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29375,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29377,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
29379,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
672659,2021-03-15,Arkansas,Arkansas,5001,2007,35.00,5000,Arkansas,AR,Arkansas,State,3017804
672661,2021-03-16,Arkansas,Arkansas,5001,2013,35.00,5000,Arkansas,AR,Arkansas,State,3017804
672663,2021-03-17,Arkansas,Arkansas,5001,2017,35.00,5000,Arkansas,AR,Arkansas,State,3017804
672665,2021-03-18,Arkansas,Arkansas,5001,2020,35.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1088311,2021-03-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1088312,2021-03-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1088313,2021-03-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1088314,2021-03-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
418,2021-03-14,Snohomish,Washington,31617,548.00,53061,WA,County,822083,3845.96,66.66
839,2021-03-14,Cook,Illinois,482973,10069.00,17031,IL,County,5150233,9377.69,195.51
1259,2021-03-14,Orange,California,263823,4465.00,6059,CA,County,3175692,8307.58,140.60
1678,2021-03-14,Maricopa,Arizona,520487,9437.00,4013,AZ,County,4485414,11603.99,210.39
2097,2021-03-14,Los Angeles,California,1210920,22474.00,6037,CA,County,10039107,12062.03,223.86
...,...,...,...,...,...,...,...,...,...,...,...
1087802,2021-03-14,Wheeler,Oregon,23,1.00,41069,OR,County,1332,1726.73,75.08
1087960,2021-03-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1088087,2021-03-14,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1088210,2021-03-14,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
418,2021-03-14,Snohomish,Washington,31617,548.00,53061,WA,County,822083,3845.96,66.66,66.66,3845.96
839,2021-03-14,Cook,Illinois,482973,10069.00,17031,IL,County,5150233,9377.69,195.51,195.51,9377.69
1259,2021-03-14,Orange,California,263823,4465.00,6059,CA,County,3175692,8307.58,140.60,140.60,8307.58
1678,2021-03-14,Maricopa,Arizona,520487,9437.00,4013,AZ,County,4485414,11603.99,210.39,210.39,11603.99
2097,2021-03-14,Los Angeles,California,1210920,22474.00,6037,CA,County,10039107,12062.03,223.86,223.86,12062.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087802,2021-03-14,Wheeler,Oregon,23,1.00,41069,OR,County,1332,1726.73,75.08,75.08,1726.73
1087960,2021-03-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1088087,2021-03-14,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1088210,2021-03-14,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
765320,2021-03-14,Clarke,Alabama,3464,56.00,1025,AL,County,23622,14664.30,237.07,237.07,14664.30,1
703275,2021-03-14,Hale,Alabama,2136,72.00,1065,AL,County,14651,14579.21,491.43,491.43,14579.21,2
550576,2021-03-14,Lowndes,Alabama,1342,52.00,1085,AL,County,9726,13798.07,534.65,534.65,13798.07,3
488302,2021-03-14,Etowah,Alabama,13496,335.00,1055,AL,County,102268,13196.70,327.57,327.57,13196.70,4
550935,2021-03-14,Marengo,Alabama,2470,58.00,1091,AL,County,18863,13094.42,307.48,307.48,13094.42,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671237,2021-03-14,Converse,Wyoming,972,17.00,56009,WY,County,13822,7032.27,122.99,122.99,7032.27,19
671594,2021-03-14,Sublette,Wyoming,681,7.00,56035,WY,County,9831,6927.07,71.20,71.20,6927.07,20
833962,2021-03-14,Lincoln,Wyoming,1278,12.00,56023,WY,County,19830,6444.78,60.51,60.51,6444.78,21
927738,2021-03-14,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
550576,2021-03-14,Lowndes,Alabama,1342,52.00,1085,AL,County,9726,13798.07,534.65,534.65,13798.07,3,1
703275,2021-03-14,Hale,Alabama,2136,72.00,1065,AL,County,14651,14579.21,491.43,491.43,14579.21,2,2
234527,2021-03-14,Walker,Alabama,6689,268.00,1127,AL,County,63521,10530.38,421.91,421.91,10530.38,31,3
486862,2021-03-14,Clay,Alabama,1469,54.00,1027,AL,County,13235,11099.36,408.01,408.01,11099.36,20,4
549858,2021-03-14,Crenshaw,Alabama,1484,55.00,1041,AL,County,13772,10775.49,399.36,399.36,10775.49,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592220,2021-03-14,Hot Springs,Wyoming,362,3.00,56017,WY,County,4413,8203.04,67.98,67.98,8203.04,17,19
833962,2021-03-14,Lincoln,Wyoming,1278,12.00,56023,WY,County,19830,6444.78,60.51,60.51,6444.78,21,20
813253,2021-03-14,Uinta,Wyoming,2088,12.00,56041,WY,County,20226,10323.35,59.33,59.33,10323.35,4,21
233429,2021-03-14,Teton,Wyoming,3427,9.00,56039,WY,County,23464,14605.35,38.36,38.36,14605.35,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085749,2021-03-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1085750,2021-03-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1085751,2021-03-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1085752,2021-03-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
762411,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,237.07,14664.30,37,1,2.00
762412,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,237.07,14664.30,37,1,0.00
762413,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,237.07,14664.30,37,1,1.00
762414,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,237.07,14664.30,37,1,4.00
762415,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,237.07,14664.30,37,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847851,2021-03-15,Crook,Wyoming,419,11.00,56011,WY,County,7584,5524.79,145.04,145.04,5511.60,7,23,1.00
847852,2021-03-16,Crook,Wyoming,419,11.00,56011,WY,County,7584,5524.79,145.04,145.04,5511.60,7,23,0.00
847853,2021-03-17,Crook,Wyoming,419,11.00,56011,WY,County,7584,5524.79,145.04,145.04,5511.60,7,23,0.00
847854,2021-03-18,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5511.60,7,23,3.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
548375,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13798.07,1,3,1.00,0.00
548376,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13798.07,1,3,0.00,0.00
548377,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13798.07,1,3,0.00,0.00
548378,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13798.07,1,3,0.00,0.00
548379,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13798.07,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546575,2021-03-15,Albany,Wyoming,3931,11.00,56001,WY,County,38880,10110.60,28.29,28.29,10095.16,23,6,6.00,0.00
546576,2021-03-16,Albany,Wyoming,3931,11.00,56001,WY,County,38880,10110.60,28.29,28.29,10095.16,23,6,0.00,0.00
546577,2021-03-17,Albany,Wyoming,3934,11.00,56001,WY,County,38880,10118.31,28.29,28.29,10095.16,23,6,3.00,0.00
546578,2021-03-18,Albany,Wyoming,3936,11.00,56001,WY,County,38880,10123.46,28.29,28.29,10095.16,23,6,2.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
279879,2021-03-14,Imperial,California,27099,677.00,6025,CA,County,181215,14954.06,373.59,373.59,14954.06,1,2,28.00,0.00
2097,2021-03-14,Los Angeles,California,1210920,22474.00,6037,CA,County,10039107,12062.03,223.86,223.86,12062.03,2,5,634.00,28.00
489694,2021-03-14,Inyo,California,1344,35.00,6027,CA,County,18039,7450.52,194.02,194.02,7450.52,3,24,5.00,0.00
139525,2021-03-14,San Bernardino,California,288787,3972.00,6071,CA,County,2180085,13246.59,182.19,182.19,13246.59,4,4,188.00,55.00
62468,2021-03-14,Stanislaus,California,57504,969.00,6099,CA,County,550660,10442.74,175.97,175.97,10442.74,5,9,0.00,0.00
77425,2021-03-14,Tulare,California,48550,798.00,6107,CA,County,466195,10414.10,171.17,171.17,10414.10,6,10,0.00,0.00
28987,2021-03-14,Riverside,California,291866,3971.00,6065,CA,County,2470546,11813.83,160.73,160.73,11813.83,7,6,0.00,0.00
49720,2021-03-14,San Joaquin,California,67817,1210.00,6077,CA,County,762148,8898.14,158.76,158.76,8898.14,8,19,0.00,0.00
593595,2021-03-14,Kings,California,22378,235.00,6031,CA,County,152940,14631.88,153.66,153.66,14631.88,9,3,25.00,0.00
28231,2021-03-14,Fresno,California,97324,1527.00,6019,CA,County,999101,9741.16,152.84,152.84,9741.16,10,14,139.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1033292,2021-03-14,Lassen,California,5629,1.00,6035,CA,County,30573,18411.67,3.27,3.27,18411.67,56,1,0.00,0.00
279879,2021-03-14,Imperial,California,27099,677.00,6025,CA,County,181215,14954.06,373.59,373.59,14954.06,1,2,28.00,0.00
593595,2021-03-14,Kings,California,22378,235.00,6031,CA,County,152940,14631.88,153.66,153.66,14631.88,9,3,25.00,0.00
139525,2021-03-14,San Bernardino,California,288787,3972.00,6071,CA,County,2180085,13246.59,182.19,182.19,13246.59,4,4,188.00,55.00
2097,2021-03-14,Los Angeles,California,1210920,22474.00,6037,CA,County,10039107,12062.03,223.86,223.86,12062.03,2,5,634.00,28.00
28987,2021-03-14,Riverside,California,291866,3971.00,6065,CA,County,2470546,11813.83,160.73,160.73,11813.83,7,6,0.00,0.00
175360,2021-03-14,Kern,California,104878,1140.00,6029,CA,County,900202,11650.50,126.64,126.64,11650.50,14,7,64.00,8.00
443864,2021-03-14,Merced,California,29796,421.00,6047,CA,County,277680,10730.34,151.61,151.61,10730.34,11,8,0.00,0.00
62468,2021-03-14,Stanislaus,California,57504,969.00,6099,CA,County,550660,10442.74,175.97,175.97,10442.74,5,9,0.00,0.00
77425,2021-03-14,Tulare,California,48550,798.00,6107,CA,County,466195,10414.10,171.17,171.17,10414.10,6,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
279879,2021-03-14,Imperial,California,27099,677.00,6025,CA,County,181215,14954.06,373.59,373.59,14954.06,1,2,28.00,0.00
2097,2021-03-14,Los Angeles,California,1210920,22474.00,6037,CA,County,10039107,12062.03,223.86,223.86,12062.03,2,5,634.00,28.00
489694,2021-03-14,Inyo,California,1344,35.00,6027,CA,County,18039,7450.52,194.02,194.02,7450.52,3,24,5.00,0.00
139525,2021-03-14,San Bernardino,California,288787,3972.00,6071,CA,County,2180085,13246.59,182.19,182.19,13246.59,4,4,188.00,55.00
62468,2021-03-14,Stanislaus,California,57504,969.00,6099,CA,County,550660,10442.74,175.97,175.97,10442.74,5,9,0.00,0.00
77425,2021-03-14,Tulare,California,48550,798.00,6107,CA,County,466195,10414.10,171.17,171.17,10414.10,6,10,0.00,0.00
28987,2021-03-14,Riverside,California,291866,3971.00,6065,CA,County,2470546,11813.83,160.73,160.73,11813.83,7,6,0.00,0.00
49720,2021-03-14,San Joaquin,California,67817,1210.00,6077,CA,County,762148,8898.14,158.76,158.76,8898.14,8,19,0.00,0.00
593595,2021-03-14,Kings,California,22378,235.00,6031,CA,County,152940,14631.88,153.66,153.66,14631.88,9,3,25.00,0.00
28231,2021-03-14,Fresno,California,97324,1527.00,6019,CA,County,999101,9741.16,152.84,152.84,9741.16,10,14,139.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,373.59,14954.06,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3662,03/15/21,Lassen,5631,1.00,18418.21,3.27,3.27,18411.67,56,1,2.00,0.00
3663,03/16/21,Lassen,5632,1.00,18421.48,3.27,3.27,18411.67,56,1,1.00,0.00
3664,03/17/21,Lassen,5632,1.00,18421.48,3.27,3.27,18411.67,56,1,0.00,0.00
3665,03/18/21,Lassen,5634,1.00,18428.02,3.27,3.27,18411.67,56,1,2.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,373.59,14954.06,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,373.59,14954.06,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3745,03/15/21,Fresno,97461,1527.00,9754.87,152.84,152.84,9741.16,10,14,137.00,0.00
3746,03/16/21,Fresno,97520,1527.00,9760.77,152.84,152.84,9741.16,10,14,59.00,0.00
3747,03/17/21,Fresno,97638,1527.00,9772.59,152.84,152.84,9741.16,10,14,118.00,0.00
3748,03/18/21,Fresno,97727,1527.00,9781.49,152.84,152.84,9741.16,10,14,89.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)